# Module 2 Quiz

In [1]:
def hamming(s1, s2):
    """
    Find Hamming distance between two strings
    """
    dist = 0
    for i in range(len(s1)):
        if s1[i] != s2[i]:
            dist += 1
    return dist

In [2]:
s1 = "CTTGAAGTGGACCTCTAGTTCCTCTACAAAGAACAGGTTGACCTGTCGCGAAG"
s2 = "ATGCCTTACCTAGATGCAATGACGGACGTATTCCTTTTGCCTCAACGGCTCCT"

hamming(s1, s2)

43

In [4]:
def find_maximum_skew(string):
    index = 0
    skew = 0
    maximum = 0
    max_indices = []
    for i in string:
        if i == "G":
            skew += 1
        elif i == "C":
            skew -= 1
        index += 1
        
        if skew > maximum:
            max_indices = []
            max_indices.append(index)
            maximum = skew
        elif skew == maximum:
            max_indices.append(index)
    return max_indices 

In [5]:
string = "CATTCCAGTACTTCATGATGGCGTGAAGA"
find_maximum_skew(string)

[28, 29]

In [16]:
def maxmap(input_map):
    """
    Returns the maximum value (not key) in a map.

    Args:
        input_map (dict): the map (type(value) == int)

    Returns:
        max_value (int): the maximum value.
    """
    max_value = max(input_map.values())
    return max_value

def frequent_words_with_mismatches(text, k, d):
    """
    text == text
    k == length of k-mer
    d == allowed number of mismatches
    *** most frequent word does not have to be in the text itself
    """

    patterns = []
    freq_map = {}

    n = len(text)

    for i in range(n-k+1):
        pattern = text[i:i+k]
        neighborhood = neighbors(pattern, d)
        for j in range(len(neighborhood)):
            neighbor = neighborhood[j]
            if neighbor not in freq_map.keys():
                freq_map[neighbor] = 1
            else:
                freq_map[neighbor] += 1
    m = maxmap(freq_map)
    print(freq_map["AA"])

    for pattern, value in freq_map.items():
        if value == m:
            patterns.append(pattern)

    return patterns

In [17]:
frequent_words_with_mismatches("TACGCATTACAAAGCACA", 2, 1)

13


['AA']

In [7]:
def neighbors(pattern, d):
    # handle edge cases first
    if d == 0:
        return [pattern] # absolutely necessary to return list
    if len(pattern) == 1:
        return ["A", "T", "G", "C"]
    neighborhood = []
    suffix_neighbors = neighbors(pattern[1:], d)
    for text in suffix_neighbors:
        if hamming(pattern[1:], text) < d:
            for nt in ["A", "T", "G", "C"]:
                neighborhood.append(nt+text)
        else:
            neighborhood.append(pattern[0]+text)
    return neighborhood

In [9]:
len(neighbors("CCAGTCAATG", 1))

31